In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from distfit import distfit
from scipy.stats import norm
from tqdm import tqdm
from scipy.signal import savgol_filter
from rdp import rdp

def process_particle_data(input_csv, output_csv, polar_plot_path,
                          min_angle, PIXEL_TO_MICRON, window_length, polyorder, eps1):
    """
    Process particle data to calculate turns and generate a rose plot of turn angles.

    Parameters:
    - input_csv (str): Path to the input CSV file containing particle data.
    - output_csv (str): Path to save the resulting turns-per-cell data as a CSV file.
    - polar_plot_path (str): Path to save the rose plot of turn angles.
    - min_angle (float): Minimum angle threshold for detecting turns.
    - PIXEL_TO_MICRON (float): Conversion factor from pixels to microns.
    - window_length (int): Window length for Savitzky-Golay filter.
    - polyorder (int): Polynomial order for Savitzky-Golay filter.
    - eps1 (float): Epsilon for RDP algorithm.

    Returns:
    None
    """
    # Load the linked particle dataset
    df = pd.read_csv(input_csv)

    # Initialize dictionaries to store results
    turns_per_cell = {}
    all_peak_angles = []

    # Function to calculate angles between vectors
    def angle(directions):
        vec2 = directions[1:]
        vec1 = directions[:-1]
        norm1 = np.sqrt((vec1 ** 2).sum(axis=1))
        norm2 = np.sqrt((vec2 ** 2).sum(axis=1))
        cos_angle = (vec1 * vec2).sum(axis=1) / (norm1 * norm2)
        angles = np.arccos(np.clip(cos_angle, -1.0, 1.0)) * 57.2958
        cross_product = vec1[:, 0] * vec2[:, 1] - vec1[:, 1] * vec2[:, 0]
        angles[cross_product < 0] = 360 - angles[cross_product < 0]
        return angles

    # Detect turns based on angle threshold
    def detect_turns(trajectory_x, trajectory_y, min_angle):
        directions = np.diff(np.vstack((trajectory_x, trajectory_y)).T, axis=0)
        theta = angle(directions)
        idx = np.where(theta > min_angle)[0] + 1
        return idx, theta

    # Process each cell (particle) individually
    for particle_id, particle_data in tqdm(df.groupby('particle'), desc='Processing cells'):
        x = particle_data['centroid_x'].to_numpy()
        y = particle_data['centroid_y'].to_numpy()
        x_smooth = savgol_filter(x, window_length, polyorder)
        y_smooth = savgol_filter(y, window_length, polyorder)
        simplified_trajectory = rdp(np.vstack((x_smooth, y_smooth)).T, epsilon=eps1)
        sx, sy = simplified_trajectory.T
        turn_indices, turn_angles = detect_turns(sx, sy, min_angle)
        number_of_turns = len(turn_indices) or 1
        distances = np.sqrt(np.diff(x)**2 + np.diff(y)**2) * PIXEL_TO_MICRON
        total_distance = np.sum(distances)
        turns_per_cell[particle_id] = {
            "number_of_turns": number_of_turns,
            "total_distance": total_distance
        }
        valid_indices = turn_indices[turn_indices < len(turn_angles)]
        all_peak_angles.extend(turn_angles[valid_indices])

    # Save turns per cell data
    turns_per_cell_df = pd.DataFrame.from_dict(turns_per_cell, orient='index')
    turns_per_cell_df.index.name = 'particle'
    turns_per_cell_df.to_csv(output_csv)
    print(f"Saved turns per cell data to {output_csv}")

        # Generate rose plot
    all_peak_angles_rad = np.deg2rad([angle % 360 for angle in all_peak_angles if 0 <= angle <= 360])
    num_bins = 20
    bins = np.linspace(0, 2 * np.pi, num_bins + 1)
    counts, bin_edges = np.histogram(all_peak_angles_rad, bins=bins)
    
    # Combine the first and last bins for circular data
    counts[0] += counts[-1]
    counts = counts[:-1]  # Remove the redundant last bin
    
    # Adjust bin centers to match counts
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # Calculate bin centers
    bin_centers = bin_centers[:len(counts)]  # Align with `counts`
    
    plt.figure(figsize=(8, 8))
    ax = plt.subplot(111, polar=True)
    bars = ax.bar(bin_centers, counts, width=(2 * np.pi / num_bins), color="#6C757D", alpha=0.6, edgecolor="k")
    ax.set_theta_zero_location("W")
    ax.set_theta_direction(-1)
    ax.set_xticks(np.linspace(0, 2 * np.pi, num_bins, endpoint=False))
    ax.set_xticklabels([f"{int(np.degrees(angle))}°" for angle in np.linspace(0, 2 * np.pi, num_bins, endpoint=False)], fontsize=20)
    for label in ax.get_xticklabels():
        label.set_y(label.get_position()[1] - 0.1)
    ax.set_yticklabels([f"{int(tick)}" for tick in ax.get_yticks()], fontsize=20)
    plt.savefig(polar_plot_path, dpi=300)
    plt.show()






def analyze_particle_characteristics(df, PIXEL_TO_MICRON, FPS):
    """
    Analyze particle characteristics including major axis, minor axis, aspect ratio, and speed.

    Parameters:
    - df (DataFrame): The particle data DataFrame.
    - PIXEL_TO_MICRON (float): Conversion factor for pixels to microns.
    - FPS (int): Frames per second of the tracking data.

    Returns:
    None
    """
    q = df.particle.unique()
    mean_speeds = []
    mean_lengths = []
    mean_minor_lengths = []
    mean_aspect_ratios = []

    for ii in tqdm(q, desc='Processing particles'):
        
        track = df.loc[df.particle == ii, ['frame', 'x', 'y', 'major_axis_length', 'minor_axis_length', 'particle']]
        major_length_in_um = track["major_axis_length"].to_numpy() * PIXEL_TO_MICRON
        minor_length_in_um = track["minor_axis_length"].to_numpy() * PIXEL_TO_MICRON
        aspect_ratio = major_length_in_um / minor_length_in_um

        major_length_fit = distfit(distr='norm')
        major_length_fit.fit_transform(major_length_in_um)
        minor_length_fit = distfit(distr='norm')
        minor_length_fit.fit_transform(minor_length_in_um)

        plt.figure()
        plt.hist(major_length_in_um, bins=10, color="#D35400", edgecolor="black", alpha=0.7, label="Major Axis")
        plt.hist(minor_length_in_um, bins=10, color="#3498DB", edgecolor="black", alpha=0.7, label="Minor Axis")
        plt.title(f"Cell Length Distribution for Particle {ii}")
        plt.xlabel("Cell Length (µm)")
        plt.ylabel("Frequency")
        plt.legend()
        plt.show()

        mean_major_length = major_length_fit.model['loc']
        mean_minor_length = minor_length_fit.model['loc']
        mean_lengths.append(mean_major_length)
        mean_minor_lengths.append(mean_minor_length)
        mean_aspect_ratios.append(np.mean(aspect_ratio))

        x = track["x"].to_numpy()
        y = track["y"].to_numpy()
        x1 = np.diff(x)
        y1 = np.diff(y)
        dist = np.sqrt(x1**2 + y1**2) * PIXEL_TO_MICRON
        time_intervals = np.diff(track["frame"].to_numpy()) * (1 / FPS)
        speed = dist / time_intervals
        speed_fit = distfit(distr='norm')
        speed_fit.fit_transform(speed)

        plt.figure()
        plt.hist(speed, bins=10, color="#2E86C1", edgecolor="black", alpha=0.7)
        plt.title(f"Speed Distribution for Particle {ii}")
        plt.xlabel("Speed (µm/s)")
        plt.ylabel("Frequency")
        speed_fit.plot()
        plt.show()

        mean_speed = speed_fit.model['loc']
        mean_speeds.append(mean_speed)

    # Save mean aspect ratios to a CSV
    aspect_ratio_df = pd.DataFrame(mean_aspect_ratios, columns=['Aspect Ratio'])
    aspect_ratio_df.to_csv('mean_aspect_ratios.csv', index=False)
    print("Saved mean aspect ratios to 'mean_aspect_ratios.csv'")

    overall_mean_aspect_ratio, overall_std_aspect_ratio = norm.fit(mean_aspect_ratios)
    plt.figure()
    plt.hist(mean_aspect_ratios, bins=10, color="#9B59B6", edgecolor="black", alpha=0.7, density=True)
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, overall_mean_aspect_ratio, overall_std_aspect_ratio)
    plt.plot(x, p, 'k--', linewidth=2)
    plt.title("Overall Mean Aspect Ratio Distribution (Per Particle)")
    plt.xlabel("Aspect Ratio (Major/Minor)")
    plt.ylabel("Density")
    plt.text(xmin + 0.05 * (xmax - xmin), max(p) * 0.9, f"Mean = {overall_mean_aspect_ratio:.2f}", fontsize=12)
    plt.savefig("Overall_Mean_Aspect_Ratio_Per_Particle.png", dpi=300, bbox_inches="tight")
    plt.close()

    print("Plots saved as 'Overall_Mean_Aspect_Ratio_Per_Particle.png'")

# Example usage
process_particle_data(
    input_csv='linked_particles_flavo_case.csv',
    output_csv='turns_per_cell_data.csv',
    polar_plot_path='overall_turn_angle_distributions.png',
    min_angle=1,
    PIXEL_TO_MICRON=0.166666666666666,
    window_length=5,
    polyorder=2,
    eps1=5
)

analyze_particle_characteristics(
    df=pd.read_csv('linked_particles_flavo_case.csv'),
    PIXEL_TO_MICRON=0.166666666666666,
    FPS=15
)
